In [ ]:
# ============================================================
# RadDet ICASSP 2025 Radar Spectrum Detection Dataset
# Cell 1: Download, extract, visualize, EDA
# ============================================================

import os
import tarfile
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

import kagglehub

# ------------------------------------------------------------
# Step 1: Download Kaggle dataset
# ------------------------------------------------------------
print("Downloading RadDet dataset from Kaggle...")
dataset_path = kagglehub.dataset_download("abcxyzi/raddet-icassp-2025")
print("Dataset downloaded to:", dataset_path)

# The downloaded directory will contain ~21 multi-part tar files.
# We reassemble each dataset according to its naming pattern.

parts = sorted(glob.glob(os.path.join(dataset_path, "*.part-*")))
print("Found multipart archives:", len(parts))

# ------------------------------------------------------------
# Step 2: Auto-recombine all multi-part tar files
# ------------------------------------------------------------

def recombine_tar(prefix):
    """Recombine all .part-* for a given prefix.tar.part-*"""
    part_files = sorted(glob.glob(os.path.join(dataset_path, prefix + ".part-*")))
    if not part_files:
        return None

    combined = os.path.join(dataset_path, prefix + ".tar")
    if os.path.exists(combined):
        return combined

    print(f"Recombining: {prefix}")
    with open(combined, "wb") as outfile:
        for p in part_files:
            with open(p, "rb") as infile:
                outfile.write(infile.read())
    return combined

# Detect all unique prefixes
unique_prefixes = sorted(
    set(os.path.basename(p).split(".part-")[0] for p in parts)
)

combined_archives = []
for prefix in unique_prefixes:
    t = recombine_tar(prefix)
    if t is not None:
        combined_archives.append(t)

print("Combined archives:", combined_archives)

# ------------------------------------------------------------
# Step 3: Extract each tar (will create images/, labels/, etc.)
# ------------------------------------------------------------
extract_dir = os.path.join(dataset_path, "extracted")
os.makedirs(extract_dir, exist_ok=True)

for t in combined_archives:
    print("Extracting:", t)
    with tarfile.open(t, "r") as tar:
        tar.extractall(extract_dir)

print("Extraction complete.")

# ------------------------------------------------------------
# Step 4: EDA – Inspect sample images and draw bbox overlays
# ------------------------------------------------------------

# Locate PNGs and YOLO label files
pngs = sorted(glob.glob(os.path.join(extract_dir, "**", "*.png"), recursive=True))
txts = [p.replace(".png", ".txt") for p in pngs]

print("Sample count:", len(pngs))

def load_yolo_bboxes(label_file):
    """Load YOLO (class cx cy w h) format."""
    if not os.path.exists(label_file):
        return []
    boxes = []
    with open(label_file, "r") as f:
        for line in f:
            vals = line.strip().split()
            cls = int(vals[0])
            cx, cy, w, h = map(float, vals[1:])
            boxes.append((cls, cx, cy, w, h))
    return boxes

def draw_boxes(im, bboxes, color=(255,0,0)):
    """Draw YOLO-format boxes on image."""
    h, w = im.shape[:2]
    overlay = im.copy()
    for cls, cx, cy, bw, bh in bboxes:
        x1 = int((cx - bw/2) * w)
        y1 = int((cy - bh/2) * h)
        x2 = int((cx + bw/2) * w)
        y2 = int((cy + bh/2) * h)
        cv2.rectangle(overlay, (x1,y1), (x2,y2), color, 1)
    return overlay

# ------------------------------------------------------------
# EDA Visualizations
# ------------------------------------------------------------
n_show = 4
plt.figure(figsize=(12, 8))

for i in range(n_show):
    img_path = pngs[i]
    lbl_path = txts[i]
    im = np.array(Image.open(img_path).convert("RGB"))
    bboxes = load_yolo_bboxes(lbl_path)
    im2 = draw_boxes(im, bboxes)

    plt.subplot(2, 2, i+1)
    plt.imshow(im2, cmap="gray")
    plt.title(f"Frame {i}, boxes={len(bboxes)}")
    plt.axis("off")

plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# Step 5: Waterfall view
# Convert spectrogram into stacked frequency slices for visualization
# ------------------------------------------------------------

def waterfall_view(im, slices=64):
    h, w = im.shape[:2]
    slice_h = h // slices
    wf = [im[i*slice_h:(i+1)*slice_h, :].mean(axis=0) for i in range(slices)]
    return np.array(wf)

sample = np.array(Image.open(pngs[0]).convert("L"))
wf = waterfall_view(sample, slices=128)

plt.figure(figsize=(10,4))
plt.imshow(wf, aspect="auto", cmap="inferno")
plt.title("Waterfall")
plt.xlabel("Frequency")
plt.ylabel("Time slice index")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# RadDet Radar Detection Baseline Training
# Cell 2: Model, Training, Evaluation
# ============================================================

import os
from ultralytics import YOLO

# You can train YOLO directly with the provided data.yaml
# Choose any of these (256 recommended as baseline speed/accuracy tradeoff)

data_yaml = os.path.join(dataset_path, "extracted", "data.yaml")
print("Using config:", data_yaml)

# ------------------------------------------------------------
# Step 1: Load YOLO model (YOLOv8n as baseline)
# ------------------------------------------------------------
model = YOLO("yolov8n.pt")

# ------------------------------------------------------------
# Step 2: Train
# ------------------------------------------------------------
results = model.train(
    data=data_yaml,
    imgsz=256,
    epochs=30,
    batch=16,
    lr0=0.001,
    device=0,
    project=os.path.join(dataset_path, "runs"),
    name="raddet_yolov8n_baseline",
)

# ------------------------------------------------------------
# Step 3: Evaluate on test set
# ------------------------------------------------------------
metrics = model.val()
print("Validation metrics:", metrics)

# ------------------------------------------------------------
# Step 4: Export as TensorFlow SavedModel or Keras
# ------------------------------------------------------------
export_dir = os.path.join(dataset_path, "yolo_export")
os.makedirs(export_dir, exist_ok=True)

model.export(format="saved_model", out=export_dir)
print("Export saved to:", export_dir)
